In [78]:
import jax.numpy as jnp
from crystalformer.src.utils import GLXYZAW_from_file
from crystalformer.src.elements import element_dict

In [6]:
alex20_folder = '/opt/data/bcmdata/ZONES/data/PROJECTS/datafile/PRIVATE/zdcao/crystal_gpt/dataset/alex/PBE/alex20/'

train_path = alex20_folder+'/train.lmdb'
valid_path = alex20_folder+'/val.lmdb'
test_path = alex20_folder+'/test.lmdb'


In [7]:
atom_types = 119 
wyck_types = 28 
n_max = 21

train_dataset = GLXYZAW_from_file(train_path, atom_types, wyck_types, n_max)
valid_dataset = GLXYZAW_from_file(valid_path, atom_types, wyck_types, n_max)
test_dataset = GLXYZAW_from_file(test_path, atom_types, wyck_types, n_max)





G: (1071694,)
L: (1071694, 6)
XYZ: (1071694, 21, 3)
A: (1071694, 21)
W: (1071694, 21)
G: (133962,)
L: (133962, 6)
XYZ: (133962, 21, 3)
A: (133962, 21)
W: (133962, 21)
G: (133962,)
L: (133962, 6)
XYZ: (133962, 21, 3)
A: (133962, 21)
W: (133962, 21)


In [9]:
G, L, XYZ, A, W = train_dataset

In [83]:
def find_element(x, elements):
    allowed_values = [0] + [element_dict[e] for e in elements]
    allowed = jnp.asarray(allowed_values)

    # eq[i, j, k] = (x[i, j] == allowed[k])
    eq = (x[:, :, None] == allowed[None, None, :])

    # (1) only allowed values: for each element, it's equal to at least one allowed
    only_allowed = jnp.all(jnp.any(eq, axis=2), axis=1)   # shape (n,)

    # (2) all allowed present: for each allowed value k, it appears in the row at least once
    all_present = jnp.all(jnp.any(eq, axis=1), axis=1)    # shape (n,)

    return jnp.where(only_allowed & all_present)[0]

In [102]:
idx = find_element(A, ['C', 'H'])



In [103]:
A[idx]

Array([[6, 6, 6, 1, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],      dtype=int32)

In [104]:
G[idx]

Array([  2,  62, 164, 217], dtype=int32)

In [105]:
W[idx]

Array([[9, 9, 9, 9, 9, 9, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [3, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],      dtype=int32)

In [106]:
L[idx]

Array([[1.0226715, 1.6489164, 6.179922 , 1.6630903, 1.6472576, 1.5715139],
       [3.358569 , 1.1186354, 2.2955177, 1.5707964, 1.5707964, 1.5707964],
       [1.6003149, 1.6003149, 3.2431977, 1.5707964, 1.5707964, 2.0943952],
       [2.2398098, 2.2398098, 2.2398098, 1.5707964, 1.5707964, 1.5707964]],      dtype=float32)